GETTING THE AUDIO DEVICE INFO

In [ ]:
import pyaudio

p=pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()

The main code

In [5]:
import pyaudio
import subprocess
import json
from vosk import Model, KaldiRecognizer
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
from queue import Queue

CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 2
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

model = Model(model_name="vosk-model-small-en-us-0.15")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

messages = Queue()
recordings = Queue()


def record_microphone(chunk=512):
    p = pyaudio.PyAudio()
    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate=FRAME_RATE,
                    input=True,
                    input_device_index=2,  
                    frames_per_buffer=chunk)
    
    frames = []
    
    
    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        
        
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames = []  

    stream.stop_stream()
    stream.close()
    p.terminate()


def speech_recognition(output):
    while not messages.empty():
        frames = recordings.get()
        print("Received frames for transcription")
        
        if rec.AcceptWaveform(b''.join(frames)):
            result = rec.Result()
            text = json.loads(result)["text"]
            print("Recognized TEXT:==>", text)
            
           
            with output:
                output.append_stdout(f"Recognized: {text}\n")
            
            cased = subprocess.check_output("python recasepunc/recasepunc.py predict recasepunc/checkpoint", shell=True, text=True, input=text)
            with output:
                output.append_stdout(f"Cased: {cased}\n")

record_button = widgets.Button(description="Record", disabled=False, button_style="success", icon="microphone")
stop_button = widgets.Button(description="Stop", disabled=False, button_style="warning", icon="stop")
output = widgets.Output()


def start_recording(data):
    messages.put(True)
    with output:
        display("Starting...")
        record_thread = Thread(target=record_microphone)
        record_thread.start()
        transcribe_thread = Thread(target=speech_recognition, args=(output,))
        transcribe_thread.start()


def stop_recording(data):
    while not messages.empty():
        messages.get() 
    with output:
        display("Stopped.")


record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)


Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()